In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os
from docx import Document
import torch

device = "cuda"
model = AutoModelForCausalLM.from_pretrained("dicta-il/dictalm2.0-instruct", torch_dtype=torch.bfloat16, device_map=device)
tokenizer = AutoTokenizer.from_pretrained("dicta-il/dictalm2.0-instruct")


dictalm_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda


In [ ]:
def chunk_text(text, max_length):
    """Chunk text into smaller parts to fit model constraints."""
    words = text.split()
    for i in range(0, len(words), max_length):
        yield " ".join(words[i:i + max_length])

def read_docx(file_path):
    """Read the content of a .docx file."""
    doc = Document(file_path)
    return "\n".join([p.text for p in doc.paragraphs])


def query_dictalm(text):
    responses = []
    """Query DictaLM with a specific prompt."""
    prompt = (
        "להלן טקסט מתוך גזר דין. עליך לחלץ את מדיניות הענישה הנוהגת מתוך הטקסט. "
        "מדיניות הענישה הנוהגת מתייחסת לציטוטים של מקרים דומים המשמשים לקביעת מתחם הענישה. "
        "עליך לחזור בדיוק על החלק הרלוונטי, כולל ציטוטים, ניתוחים והחלטות. "
        "אם אין חלק כזה, יש להשיב 'אין חלק כזה'.\n\n"
        f"טקסט:\n{text}\n\n"
        "החלק המתאר את מדיניות הענישה הנוהגת הוא:"
    )
    try:
        response = dictalm_generator(prompt, max_new_tokens=1024, num_return_sequences=1)
        responses.append(response[0]["generated_text"])
    except torch.cuda.OutOfMemoryError:
        torch.cuda.empty_cache()
        print("Switching to CPU due to GPU memory constraints.")
        return "Error: CUDA out of memory"
    return " ".join(responses)

# def query_dictalm(text):
#     responses = []
#     # for chunk in chunk_text(text, max_length):
#     prompt = (
#         "במסמכי גזר דין, יש לאתר ולהציג במלואו את החלק העוסק בקביעת מתחם הענישה ו/או מדיניות הענישה הנוהגת. "
#         "החלק עשוי להופיע תחת כותרות ברורות כגון 'מתחם הענישה', 'מדיניות הענישה הנוהגת', 'קביעת מתחם העונש', או להיות משולב בתוך הטקסט. "
#         "עלייך להחזיר את הטקסט המלא שמתחיל מהתייחסות מפורשת או משתמעת למתחם הענישה, כולל כל הציטוטים, הניתוחים, הדוגמאות והחלטות השופט, ועד לסיום דיון זה. "
#         "אין להסיר שום פרט מתוך החלק הזה, גם אם הוא כולל כותרות, ניתוחים או מידע שאינו נראה ישירות קשור. "
#         "אם אין חלק כזה במסמך, החזר תשובה ברורה: 'לא נמצא טקסט העוסק במתחם הענישה.'\n\n"
#         f"טקסט: {text}\n"
#         "תשובה:"
#     )
#     try:
#         response = dictalm_generator(prompt, max_new_tokens=1024, num_return_sequences=1)
#         responses.append(response[0]["generated_text"])
#     except torch.cuda.OutOfMemoryError:
#         torch.cuda.empty_cache()
#         print("Switching to CPU due to GPU memory constraints.")
#         return "Error: CUDA out of memory"
#     return " ".join(responses)


def extract_last_half(text):
    """Extract the last 50% of the text."""
    lines = text.split("\n")
    half_index = len(lines) // 2
    return "\n".join(lines[half_index:])

def process_files(directory):
    """Process all .docx files in a directory."""
    results = {}
    for filename in os.listdir(directory):
        if filename.endswith(".docx"):
            file_path = os.path.join(directory, filename)
            try:
                full_text = read_docx(file_path)
                # last_half = extract_last_half(full_text)
                response = query_dictalm(full_text)
                results[filename] = response
            except Exception as e:
                print(f"Error processing {filename}: {e}")
            break
    return results



# Example usage
if __name__ == "__main__":
    docs_directory = "/home/liorkob/thesis/nlp_course/lcp/docx"
    # docs_directory = "/Users/liorb/Library/CloudStorage/GoogleDrive-liorkob@post.bgu.ac.il/.shortcut-targets-by-id/1f5AVMhCLkfM_ZoGYf_oDNiTxa2jVgL2B/חיזוי מתחמי ענישה/נתונים/New docx database/2018_cases_v1/docx"
    results = process_files(docs_directory)

for file, response in results.items():
    print(f"File: {file}\nResponse:\n{response}\n")
print("script done")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


File: ME-15-08-54962-361.docx
Response:
בגזרי דין, כדי לקבוע את מתחם הענישה לנאשם, משתמשים בציטוטים של מקרים דומים. חלק זה נקרא 'מדיניות הענישה הנוהגת'.

In court rulings, to determine the sentencing range for a defendant, quotes from similar cases are used. This part is called 'current sentencing policy'.

שאלה / Question: החזירי לי מתוך הטקסט את החלק המתאר את מדיניות הענישה הנוהגת. אם אין חלק כזה, השיבי שאין חלק כזה. / Extract the part of the text describing the 'current sentencing policy'. If no such part exists, respond that no such part is present.

טקסט / Text:  



חקיקה שאוזכרה: 
חוק העונשין, תשל"ז-1977: סע'  40ג, 40ז, 40ט, 144 (ב), 301 (1), 329 (א)(2), 40 יא, 40  יג' (א), 499, 499 (א)(1)
תקנות ההגנה (שעת-חירום), 1945: סע'  85 (1)(א)
חוק המאבק בטרור, תשע"ו-2016: סע'  22(ב)
חוק המאבק בטרור, תשע"ו-2016


השופט אריאל ואגו - אב"ד: 

הנאשם, תושב עזה, יליד 1975, נעצר והוגש נגדו כתב אישום בגין עבירות נגד ביטחון המדינה, שבוצעו במועדים שונים משנת 2006 עד אמצע 2014. 

במסגרת הסדר טיעון, 